## Parameters

In [1]:
from pathlib import Path

EXP_DIR = Path("./exps/")

In [2]:
# DEVICE_CONFIGS = ["0", "1", "2", "3"]
# DEVICE_CONFIGS = ["0,1", "2,3"]
DEVICE_CONFIGS = ["0,1,2,3"]

In [3]:
common_params = {
    "params.model.path": [
        # Small models
        # "Qwen/Qwen2.5-1.5B-Instruct",
        "Qwen/Qwen2.5-7B-Instruct",
        "meta-llama/Llama-3.1-8B-Instruct",
        # "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",

        # Medium models
        "Qwen/Qwen2.5-14B-Instruct",
        # "Qwen/Qwen2.5-32B-Instruct",
        # "Qwen/Qwen2.5-Coder-32B-Instruct",
        # "meta-llama/Llama-3.1-70B-Instruct",
        # "Qwen/QwQ-32B",

        # GRPO models
        "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-musique-merged",  # Ragent-1
        "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250508_213215-merged",  # Ragent-4
        "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250520_080809-merged",  # Ragent-5
        "bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250526_110630", # Ragent-6
    ],
    "params.model.temperature": [
        0.5,
    ],
    "params.model.top_p": [
        0.95,
    ],
    "params.model.few_shot_prob": [
        0.0,
    ],
    "params.retriever.name": [
        "hybrid-tei",
    ],
    "params.retriever.top_k": [
        1,
    ],
    "params.repeat": [
        1,
        5,
    ],
    "params.run": [
        1,
        # 2,
        # 3
    ],
}

In [4]:
varying_params_list = [
    {
        "params.dataset.path": ["bdsaglam/triviaqa-wiki-musique-mini"],
        "params.dataset.name": ["default"],
        "params.dataset.split": ["validation"],
    },
]

## Setup

In [5]:
import itertools
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn

def sorted_tuple(x):
    return tuple(sorted(x))

def jprint(obj):
    print(json.dumps(obj, indent=2))

In [6]:
df = pd.DataFrame([json.loads(fp.read_text()) for fp in EXP_DIR.glob("*.json")])
len(df)

15

In [7]:
df.drop(columns=['params.devices'], inplace=True, errors='ignore')
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [8]:
if len(df):
    df['params.model.temperature'] = df['params.model.temperature'].astype(float).map(lambda x: round(x, 2))
    df['params.model.top_p'] = df['params.model.top_p'].astype(float).map(lambda x: round(x, 2))
    
    df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
    df.drop_duplicates(subset=param_cols, inplace=True, keep='last')
    
    print(f"{len(df)} experiments after preprocessing")

15 experiments after preprocessing


In [9]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/triviaqa-wiki-musique-mini', 'bdsaglam/musique-mini']

- params.dataset.name: ['default', 'answerable']

- params.dataset.split: ['validation']

- params.model.path: ['bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250526_110630', 'bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-musique-merged', 'bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250508_213215-merged', 'meta-llama/Llama-3.1-8B-Instruct', 'Qwen/Qwen2.5-7B-Instruct', 'bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250520_080809-merged', 'Qwen/Qwen2.5-14B-Instruct']

- params.model.temperature: [0.5]

- params.model.top_p: [0.95]

- params.model.few_shot_prob: [0.0]

- params.retriever.name: ['hybrid-tei']

- params.retriever.top_k: [1]

- params.repeat: [5, 1]

- params.run: [1]



## Setup remaining experiments

In [10]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [11]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [12]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

14 experiment configurations
['params.model.path', 'params.model.temperature', 'params.model.top_p', 'params.model.few_shot_prob', 'params.retriever.name', 'params.retriever.top_k', 'params.repeat', 'params.run', 'params.dataset.path', 'params.dataset.name', 'params.dataset.split']


In [13]:
def preprocess_config(config):
    config = {k: round(v, 2) if isinstance(v, float) else v for k, v in config.items()}
    if '/home/baris' in config['params.model.path']:
        config['params.model.path'] = "bdsaglam/" + config['params.model.path'].rsplit("/", 1)[-1]
    return config

In [14]:
if len(df):
    existing_configs = [preprocess_config(config) for config in df[target_params].to_dict(orient="records")]
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 15


In [15]:
# find the missing configurations
missing_configs = [
    preprocess_config(dict(kv))
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

0 missing configurations


In [16]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if "[" in str(arg_value):
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [17]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for i,exp_config in enumerate(missing_configs):
        command = make_command(exp_config)
        devices = DEVICE_CONFIGS[i % len(DEVICE_CONFIGS)]
        command += f" \\\n    -S devices='\"{devices}\"'"
        f.write(command)
        f.write("\n\n")

In [18]:
next(iter(existing_configs), None)

{'params.model.path': 'bdsaglam/Llama-3.1-8B-Instruct-ragent-grpo-20250526_110630',
 'params.model.temperature': 0.5,
 'params.model.top_p': 0.95,
 'params.model.few_shot_prob': 0.0,
 'params.retriever.name': 'hybrid-tei',
 'params.retriever.top_k': 1,
 'params.repeat': 5,
 'params.run': 1,
 'params.dataset.path': 'bdsaglam/triviaqa-wiki-musique-mini',
 'params.dataset.name': 'default',
 'params.dataset.split': 'validation'}

In [ ]:
next(iter(missing_configs), None)